In [44]:
import pickle
import json
files = "./CasRel/data/baidu/duie_train.json"
with open(files,"rb") as f:
    data = f.readlines()
index = 0
a = ['上映时间', '专业代码', '改编自', '修业年限', '成立日期', 'ondate_date', '董事长', '注册资本', '占地面积', '人口数量', '所属专辑', 'inwork_影视作品', '编剧', '歌手', '票房', '导演', '主演', 'period_number', '邮政编码', '作者', '简称', '面积', '作曲', '作词']
predicates = set()
max_leng = 0
for line in data:
    line = json.loads(line)
    spo_list = line['spo_list']
    text = line['text']
    max_leng = max(len(text),max_leng)
    for spo in spo_list:
        s = spo['subject']
        p = spo['object']['@value']
        predicate = spo['predicate']
        if ( s.isdigit() or p.isdigit() ) and predicate == a[1]:
            # print(p+"\t"+predicate+"\t"+s+"\t"+text)
            predicates.add(predicate)
            index += 1
# print(predicates)
print(max_leng)


300


In [4]:
import numpy as np
import torch


aa = np.array([[0.0349,  0.0670, -0.0612, 0.0280, -0.0222,  0.0422],
         [-1.6719,  0.1242, -0.6488, 0.3313, -1.3965, -0.0682],
         [-1.3419,  0.4485, -0.6589, 0.1420, -0.3260, -0.4795]])
bb = np.array([[-0.0658, -0.1490, -0.1684, 0.7188,  0.3129, -0.1116],
         [-0.2098, -0.2980,  0.1126, 0.9666, -0.0178,  0.1222],
         [ 0.1179, -0.4622, -0.2112, 1.1151,  0.1846,  0.4283]])
cc = np.where(aa>0)     #合并a,b两个tensor，如果a中元素大于0
dd = np.where(bb>0)


In [25]:
import torch
sub_heads = torch.tensor([2,3,2,5,2,7,6,7,8,2])
sub_tails = torch.tensor([5,7,5,2,7,3,8,7,8,7])
sub_head_mapping = torch.zeros(10,1,9)
sub_tail_mapping = torch.zeros(10,1,9)
sub_head_mapping.scatter_(-1,sub_heads.reshape(10,1,1),1)
sub_tail_mapping.scatter_(-1,sub_tails.reshape(10,1,1),1)


tensor([[[0., 0., 0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.]]])

In [26]:
fuse = sub_heads>sub_tails
fuse

tensor([False, False, False,  True, False,  True, False, False, False, False])

In [27]:
sub_tail_mapping[fuse]=0

In [47]:
np.array([1]*4)[:3]

array([1, 1, 1])